# Verifications: KNP

- Purpose: To perform morphological analysis with Juman as well as dependency analysis using KNP
- Keywords: Morphological analysis, Dependency Analysis

## 1) Jumanによる形態素解析

In [1]:
from pyknp import Juman,Jumanpp,KNP

In [2]:
# Juman インスタンスの生成
juman = Juman()

In [3]:
# サンプル文に対する形態素解析
str = "巨大なマーライオンがセントーサ島にいた"
result = juman.analysis(str)

In [4]:
# 形態素解析の結果表示
for mp in result.mrph_list():
    print("見出し:{}, 読み:{}, 原形:{}, 品詞:{}, 品詞細分類:{}, 活用型:{}, 活用形:{}, 意味情報:{}, 代表表記:{}".format(
    mp.midasi, mp.yomi, mp.genkei, mp.hinsi, mp.bunrui, mp.katuyou1, mp.katuyou2, mp.imis, mp.repname))

見出し:巨大な, 読み:きょだいな, 原形:巨大だ, 品詞:形容詞, 品詞細分類:*, 活用型:ナ形容詞, 活用形:ダ列基本連体形, 意味情報:代表表記:巨大だ/きょだいだ, 代表表記:巨大だ/きょだいだ
見出し:マーライオン, 読み:マーライオン, 原形:マーライオン, 品詞:名詞, 品詞細分類:普通名詞, 活用型:*, 活用形:*, 意味情報:自動獲得:Wikipedia Wikipedia上位語:像/ぞう, 代表表記:
見出し:が, 読み:が, 原形:が, 品詞:助詞, 品詞細分類:格助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:セントーサ, 読み:セントーサ, 原形:セントーサ, 品詞:名詞, 品詞細分類:普通名詞, 活用型:*, 活用形:*, 意味情報:自動獲得:Wikipedia Wikipediaリダイレクト:セントーサ島, 代表表記:
見出し:島, 読み:しま, 原形:島, 品詞:名詞, 品詞細分類:普通名詞, 活用型:*, 活用形:*, 意味情報:代表表記:島/しま 漢字読み:訓 地名末尾 カテゴリ:場所-自然, 代表表記:島/しま
見出し:に, 読み:に, 原形:に, 品詞:助詞, 品詞細分類:格助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:いた, 読み:いた, 原形:いる, 品詞:動詞, 品詞細分類:*, 活用型:母音動詞, 活用形:タ形, 意味情報:代表表記:射る/いる, 代表表記:射る/いる


In [5]:
# pandasを用いて、結果を表形式で再表示
import pandas as pd

mps = [(mp.midasi, mp.yomi, mp.genkei, mp.hinsi, mp.bunrui, mp.katuyou1, mp.katuyou2, mp.imis, mp.repname) for mp in result.mrph_list()]
clmns = ('見出し','読み', '原形', '品詞', '品詞細分類', '活用型', '活用形', '意味情報', '代表表記') 
pd.DataFrame(mps,columns=clmns)

,見出し,読み,原形,品詞,品詞細分類,活用型,活用形,意味情報,代表表記
0,巨大な,きょだいな,巨大だ,形容詞,*,ナ形容詞,ダ列基本連体形,代表表記:巨大だ/きょだいだ,巨大だ/きょだいだ
1,マーライオン,マーライオン,マーライオン,名詞,普通名詞,*,*,自動獲得:Wikipedia Wikipedia上位語:像/ぞう,
2,が,が,が,助詞,格助詞,*,*,NIL,
3,セントーサ,セントーサ,セントーサ,名詞,普通名詞,*,*,自動獲得:Wikipedia Wikipediaリダイレクト:セントーサ島,
4,島,しま,島,名詞,普通名詞,*,*,代表表記:島/しま 漢字読み:訓 地名末尾 カテゴリ:場所-自然,島/しま
5,に,に,に,助詞,格助詞,*,*,NIL,
6,いた,いた,いる,動詞,*,母音動詞,タ形,代表表記:射る/いる,射る/いる


## KNPを用いた係り受け関係解析

In [6]:
# KNP　インスタンスの生成
knp = KNP(option = '-tab -anaphora')

In [7]:
# 係り受け構造関係（パーシング）と結果をリスト形式で取得
result = knp.parse(str)
ph_objs = result.bnst_list()

In [8]:
# 結果を句群のリスト、それをつなげた文、親文IDの３つを含む辞書データに変換
ph_dic = {}
for ph_obj in ph_objs:
    phrase_list = []
    for mrph in ph_obj._mrph_list._mrph:
        phrase_list.append(mrph.midasi)
    ph_dic[ph_obj.bnst_id] = {"phrase_list":phrase_list, "phrase":''.join(phrase_list), "parent_id": ph_obj.parent_id}

In [9]:
# 結果（係り受け関係）を表示
for p in ph_dic.items():
    if p[1]['parent_id'] > 0:
        print(p[1]['phrase'], '=>', ph_dic[p[1]['parent_id']]['phrase'])

巨大な => マーライオンが
マーライオンが => いた
セントーサ島に => いた


In [10]:
# 結果（係り受け関係）をリスト形式で表示
for p in ph_dic.items():
    if p[1]['parent_id'] > 0:
        print(p[1]['phrase_list'], '=>', ph_dic[p[1]['parent_id']]['phrase_list'])

['巨大な'] => ['マーライオン', 'が']
['マーライオン', 'が'] => ['いた']
['セントーサ', '島', 'に'] => ['いた']


# Reference
http://nlp.ist.i.kyoto-u.ac.jp/index.php?plugin=attach&refer=KNP&openfile=knp_feature.pdf

http://fvcrc.i.nagoya-u.ac.jp/~sasano/knp/

